In [11]:
import json
import math
import string
from typing import List
import time

import numpy as np

from datamodel import Listing, Order, OrderDepth, TradingState, UserId, Trade

In [12]:
timestamp = time.time()

listings = {
	"PRODUCT1": Listing(
		symbol="PRODUCT1", 
		product="PRODUCT1", 
		denomination= "SEASHELLS"
	),
	"PRODUCT2": Listing(
		symbol="PRODUCT2", 
		product="PRODUCT2", 
		denomination= "SEASHELLS"
	),
}

order_depths = {
	"PRODUCT1": OrderDepth(
		buy_orders={10: 7, 9: 5},
		sell_orders={11: -4, 12: -8}
	),
	"PRODUCT2": OrderDepth(
		buy_orders={142: 3, 141: 5},
		sell_orders={144: -5, 145: -8}
	),	
}

own_trades = {
	"PRODUCT1": [],
	"PRODUCT2": []
}

market_trades = {
	"PRODUCT1": [
		Trade(
			symbol="PRODUCT1",
			price=11,
			quantity=4,
			buyer="",
			seller="",
			timestamp=900
		)
	],
	"PRODUCT2": []
}

observations = {}
traderData = ""

In [15]:
def adjust_for_position_breaches(results, state, fill_until_position_breach=False):
    for product, orders in results.items():
        cur_position = state.position.get(product, 0)
        print(f"Current position for product {product} is {cur_position}")
        valid_orders = []
        for order_index, order in enumerate(orders):
            if cur_position + order.quantity <  (-20):
                print(f"Short Position breach for product {product} with position {cur_position} and order {order}")
                if fill_until_position_breach:
                    order = Order(product, order.price, (-20) - cur_position)
                    orders[order_index] = order
                    valid_orders.append(order)
            elif cur_position + order.quantity > 20:
                print(f"Long Position breach for product {product} with position {cur_position} and order {order}")
                if fill_until_position_breach:
                    order = Order(product, order.price, 20 - cur_position)
                    orders[order_index] = order
                    valid_orders.append(order)
            else:
                valid_orders.append(order)
            cur_position += order.quantity
        results[product] = valid_orders
    return results

In [31]:
result = {
    "AMETHYSTS": [Order("AMETHYSTS", 80, -10), Order("AMETHYSTS", 55, 2), Order("AMETHYSTS", 90, 9), Order("AMETHYSTS", 90, -8)],
    "STARFRUIT": [Order("STARFRUIT", 100, 2), Order("STARFRUIT", 100, 10), Order("STARFRUIT", 100, 10)],
}


position = {
	"AMETHYSTS": 3,
	"STARFRUIT": -5
}


state = TradingState(
	traderData,
	timestamp,
    listings,
	order_depths,
	own_trades,
	market_trades,
	position,
	observations
)

result = adjust_for_position_breaches(result, [], state, True)

[(AMETHYSTS, 55, 2), (AMETHYSTS, 90, 9)]
[(AMETHYSTS, 90, -8), (AMETHYSTS, 80, -10)]
[(STARFRUIT, 100, 2), (STARFRUIT, 100, 10), (STARFRUIT, 100, 10)]
[]


In [30]:
def adjust_for_position_breaches_and_add_random_oders(results, lowball_price, highball_price, state, fill_until_position_breach=False):
    valid_orders = {}
    for product, orders in results.items():
        valid_orders[product] = []
        buy_position_breach = False
        sell_position_breach = False
        buy_orders = [order for order in orders if order.quantity > 0]
        buy_orders = sorted(buy_orders, key=lambda x: x.price)
        sell_orders = [order for order in orders if order.quantity < 0]
        sell_orders = sorted(sell_orders, key=lambda x: x.price, reverse=True)
        cur_position = state.position.get(product, 0)
        tmp_cur_position = cur_position
        for buy_order in buy_orders:
            if tmp_cur_position + buy_order.quantity > 20:
                buy_position_breach = True
                if fill_until_position_breach:
                    buy_order = Order(product, buy_order.price, 20 - tmp_cur_position)
                    valid_orders[product].append(buy_order)
                    tmp_cur_position += buy_order.quantity
                    break
            else:
                valid_orders[product].append(buy_order)
                tmp_cur_position += buy_order.quantity
        if not buy_position_breach:
            valid_orders[product].append(Order(product, lowball_price, 20 - tmp_cur_position))
        tmp_cur_position = cur_position
        for sell_order in sell_orders:
            if tmp_cur_position + sell_order.quantity < -20:
                sell_position_breach = True
                if fill_until_position_breach:
                    sell_order = Order(product, sell_order.price, -20 - tmp_cur_position)
                    valid_orders[product].append(sell_order)
                    tmp_cur_position += sell_order.quantity
                    break
            else:
                valid_orders[product].append(sell_order)
                tmp_cur_position += sell_order.quantity
        if not sell_position_breach:
            valid_orders[product].append(Order(product, highball_price, -20 - tmp_cur_position))
    return valid_orders
    